# Tensor Derivatives

This notebook shows how to compute symbolic tensor derivatives

## Imports

In [131]:
import sympy
import itertools
from sympy.tensor import tensor

## Defining tensor indices

### Using IndexedBase

In [5]:
mu, nu = sympy.symbols('mu nu')

In [4]:
T = sympy.IndexedBase('T')

In [6]:
T[mu, nu]

T[mu, nu]

### Using TensorHead

In [8]:
L = tensor.TensorIndexType('Lorentz', 'L')

In [9]:
mu, nu = tensor.tensor_indices('mu nu', L)

In [15]:
rho = tensor.TensorIndex('rho', L, is_up=False)

In [28]:
alpha = tensor.TensorIndex('alpha', L, is_up=False)
beta = tensor.TensorIndex('beta', L, is_up=False)

In [10]:
T = tensor.TensorHead('T', [L, L])

In [17]:
T_mu_rho = T(mu, rho)
T_mu_rho

T(mu, -rho)

In [25]:
T_mu_rho.expand()

T(mu, -rho)

In [36]:
def partial_deriv(t, idx):
    pd = tensor.TensorHead('\partial', [L])
    return pd(idx) * t

In [96]:
def christoffel(u, ll, lr):
    gamma = tensor.TensorHead('\Gamma', [L, L, L])
    
    u = tensor.TensorIndex(u.name, L, is_up=True)
    ll = tensor.TensorIndex(ll.name, L, is_up=False)
    lr = tensor.TensorIndex(lr.name, L, is_up=False)
    
#     print(u, ll, lr)
#     print(gamma(u, ll, lr).args)
    return gamma(u, ll, lr)

In [165]:
def cov_deriv(t, wrt, n=0):
    idxs = indices(t)
    
    d = partial_deriv(t, wrt)
    dummy_up = tensor.TensorIndex('lambda_{:d}'.format(n), L, is_up=True)
    dummy_dn = tensor.TensorIndex('lambda_{:d}'.format(n), L, is_up=False)
    
    for idx in idxs:
        if idx.is_up:
            d += christoffel(idx, wrt, dummy_up) * t.replace(idx, dummy_up)
        else:
            d -= christoffel(dummy_dn, wrt, idx) * t.replace(idx, dummy_dn)

    L_0_up = tensor.TensorIndex('L_{:d}'.format(n), L, is_up=True)
    L_0_dn = tensor.TensorIndex('L_{:d}'.format(n), L, is_up=False)

    d = d.replace(L_0_up, dummy_up)
    d = d.replace(L_0_dn, dummy_dn)

    return  d

In [166]:
d = cov_deriv(T_mu_rho, alpha)
d

\Gamma(mu, -alpha, -lambda_0)*T(lambda_0, -rho) + \partial(-alpha)*T(mu, -rho) + (-1)*\Gamma(lambda_0, -alpha, -rho)*T(mu, -lambda_0)

In [167]:
def indices(t):
    if isinstance(t, tensor.TensAdd):
        term_indices = [indices(x) for x in t.args]
        if not all(set(term_indices[0]) == set(term_indices[i]) for i in range(len(term_indices)) if term_indices[i]):
            print(term_indices)
            raise ValueError('Invalid Tensorial Equation, added terms must share indices: {}'.format(t))
        return indices(t.args[0])
    elif isinstance(t, tensor.TensMul):
        idxs = list(itertools.chain.from_iterable(indices(x) for x in t.args))
        # remove dummy indices
        for i in idxs:
            i_names = [k for k in idxs if k.name == i.name]
            if len(i_names) == 2:
                _ = [idxs.remove(k) for k in i_names]
        return idxs
    elif isinstance(t, tensor.Tensor):
        return list(t.indices)
    elif isinstance(t, sympy.Number):
        return []
    else:
        raise ValueError('Unsupported type {} for extracting indices {}'.format(type(t), t))

In [172]:
def riemann(v):
    return cov_deriv(cov_deriv(v, alpha), beta, n=1) - cov_deriv(cov_deriv(v, beta), alpha, n=1)

In [170]:
d = cov_deriv(cov_deriv(T_mu_rho, alpha), beta, n=1)

In [171]:
d

\Gamma(mu, -beta, -L_0)*(\Gamma(L_0, -alpha, -lambda_1)*T(lambda_1, -rho) + \partial(-alpha)*T(L_0, -rho) + (-1)*\Gamma(lambda_1, -alpha, -rho)*T(L_0, -lambda_1)) + \partial(-beta)*(\Gamma(mu, -alpha, -L_0)*T(L_0, -rho) + \partial(-alpha)*T(mu, -rho) + (-1)*\Gamma(L_0, -alpha, -rho)*T(mu, -L_0)) + (-1)*\Gamma(L_0, -beta, -alpha)*(\Gamma(mu, -L_0, -lambda_1)*T(lambda_1, -rho) + \partial(-L_0)*T(mu, -rho) + (-1)*\Gamma(lambda_1, -L_0, -rho)*T(mu, -lambda_1)) + (-1)*\Gamma(L_0, -beta, -rho)*(\Gamma(mu, -alpha, -lambda_1)*T(lambda_1, -L_0) + \partial(-alpha)*T(mu, -L_0) + (-1)*\Gamma(lambda_1, -alpha, -L_0)*T(mu, -lambda_1))